# Lesson: degree day model

Today we are going to play around a bit with the [degree day model](http://www.antarcticglaciers.org/glaciers-and-climate/numerical-ice-sheet-models/modelling-glacier-melt/) (DDM) to understand it a bit better and to learn a little bit more about logical tools in numpy/pandas.


**Spend some time to read the webpage linked above explaining how the simple degree day model works.**

We will use the AWS data at Zhadang

In [ ]:
# imports and defaults
import pandas as pd  
%matplotlib inline 
import matplotlib.pyplot as plt
import numpy as np
pd.options.display.max_rows = 14
import seaborn as sns
sns.set_style('ticks')
sns.set_context('talk')

In [ ]:
# read the data
df = pd.read_csv('data/data_Zhadang_localtime.csv', index_col=0, parse_dates=True)

Such 'temperature index' approaches can really be applied, hourly, daily, monthly or whatever, though in cryopsheric science the daily timescale is most widely used. So we resample accordingly:

In [ ]:
# resample 
df = df.resample('D').mean()

To test the model, we are concentrating on the melting season 2012, which we will define as starting in the middle of May based on the data we plotted in the previous exercise, and running to the end of the available data:

In [ ]:
df = df.loc['2012-05-15':]

How can we check if ablation is happening? Well, we can check that the surface is lowering during this period ....

In [ ]:
df['SR50'].plot(title='Surface height in m (0 = ice)');

## 1. Simplest DDM

For the simplest DDM, we are going to use a single factor for the entire melting period. The DDM formulation is very simple:

$$Melt = f \cdot PDD$$

Where PDD is the sum total of daily average temperatures above 0°C in a given time period and $f$ is a melting factor. Let's define the melt in meters of snow/ice which is melted away. Determine the unit that $f$ should have in that (quite clumsy) case.

OK, so now we need to count the PDD's over that period. One method that you already learned is following:

In [ ]:
# select all days with mean temperature above 0
seltemp = df.TEMP.loc[df.TEMP > 0]
# sum these daily mean temperatures
seltemp.sum()

For the purpose of our modelling, however, it is easier to define a new variable (PDD), which is the daily average of temperature when it is above 0°C, and zero otherwise. For this we are using the numpy function [np.where](http://docs.scipy.org/doc/numpy-1.10.1/reference/generated/numpy.where.html):

In [ ]:
df['PDD'] = np.where(df.TEMP > 0, df.TEMP, 0)
# print this if you want to look at the values

Verify that the the sum of this new PDD variable corresponds to our computation above.

In [ ]:
# by summing it we can verify that its the same as the answer above
df.PDD.sum()

Bearing in mind that our defined melt season is only 3.5 months long, this number seems quite high ... so either there are loads of positive mean temperatures, or there are some 'heatwave' days ...
To find out which we can do the following to find out what percentage of the ablation season data is above 0°C:

In [ ]:
seltemp.size / df.TEMP.size * 100

**Q: Are you surprised by the percentage of days of that experience melt?**

Now we are going to calibrate our model, i.e. compute our factor $f$. 

The data we have is actually lowering not melt, so we need the total of snow/ice lowering (in m) during this period. WE can find this using  `iloc[]` (if you dont know this look it up or ask the notebook what it does) :

In [ ]:
obs_melt = df.SR50.iloc[-1] - df.SR50.iloc[0]  
# index location [0 to (length)-1] allowing us to subtract the fist from the last value
obs_melt

Its worth noting that in most glaciological applications this ablation would be expressed differently e.g. as lowering in units of water equivalent (e.g. mm w.e.) 


Using the formula $$Melt = f \cdot PDD$$ we can calculate our melt factor as:

In [ ]:
# so looking at the formula above ...
melt_factor = obs_melt / df['PDD'].sum()
melt_factor

It is now very easy to define a variable (MELT1), which is the daily melt due to this factor:

In [ ]:
df['MELT1'] = df['PDD'] * melt_factor

**Q: Lets plot this new variable. What are we looking at?**

In [ ]:
# plot it here and label the y-axis correctly

df['MELT1'].plot();
plt.ylabel('modelled lowering [m/day]');

In order to compare our modelled melt with observations, its more useful for us to show the modelled melt as a cumulative sum so that we can compare it to the sonic range data available for the station: 

In [ ]:
df['MELT1'] = (df['PDD'] * melt_factor).cumsum()
df['MELT1'].plot();
plt.ylabel('cumulative modelled lowering [m/day]');

Note that this plot now starts at 0, but we know that 0 reference in the sonic ranger data is the ice surface and at the start of the ablation season the surface was snow above this. 

Therefore, we now add the starting snow depth to this timeseries. We select the first element of the observations array with `iloc[]` and add it to ours:

In [ ]:
df['MELT1'] = (df['PDD'] * melt_factor).cumsum() + df.SR50.iloc[0]

Done! 

**Q: So how well does out model do? Let's plot the result of our modeling approach with the sonic ranger (SR50) data to see:**

In [ ]:
# plot both series in one plot to compare them

df[['MELT1', 'SR50']].plot();

**Q: Discuss the performance of our model. Where is it performing well? Where is it performing less well? Can you tell why?**

In [ ]:
# so consider the slopes of the curves
# up until the third week of June the model does not match too well, nor in the last 2 weeks of August
# in the middle section, although there is am offset the curves are quite well matched
# if we look back at our plot of the surface height, we know:
# there was snow (i) at the beginning, and (ii) probably small snow events at the end of the record ...
# OR ... the surface height changes at the end could be related to tilting of the AWS?
# note that the way this model is calibrated means that the final surface height *will* match
# note that we are evaluating with your turning data which is bad!!

## 2. A more reasonable DDM

It is more reasonable to distinguish between snow and ice in our model. Fortunately, the person who provided the data nicely set the 0 level to the original ice surface before the ablation season.

**Q: To identify the snow that is removed during the first part of the ablation season, lets use the `np.where()` function to add a new variable IS_SNOW to the dataframe, which is equal to True when the surface is above 0 and to False otherwise.**

In [ ]:
# your answer here

df['IS_SNOW'] = np.where(df.SR50 > 0, True, False)

**Q: Now follow the recipe from the simple example above to compute the PDD and melt factor $f$ for the snowmelt period at the start of the ablation season:**

In [ ]:
# first, compute the PDD sum during the snowmelt period:
# then, compute the observed melt during this period:
# finally, compute the factor:


# first, compute the PDD sum during the snowmelt period:
pdd_snow = df.PDD.loc[df['IS_SNOW']].sum()
# then, compute the observed melt during this period:
melt_snow = - df.SR50.iloc[0]
# finally, compute the factor:
fac_snow = melt_snow / pdd_snow
fac_snow

To isolate the period when the ice is exposed we can look for when there is NOT snow by using the operator, "~", which is the logical operator for "not":

In [ ]:
# you can see the behavior by doing this:
print(np.array([True, False, True]))
print(~ np.array([True, False, True]))

**Q: Now,use this and repeat what you did for the snow covered period but for the ice covered period to get the PDD sum  and $f$ during the ice period:**

In [ ]:
# first, compute the PDD sum during the ice melt period:
# then, compute the observed melt during this period. 
# Finally, compute the factor:


# first, compute the PDD sum during the ice melt period:
pdd_ice = df.PDD.loc[~ df['IS_SNOW']].sum()  # note the ~
# then, compute the observed melt during this period. 
melt_ice = df.SR50.iloc[-1]
# Finally, compute the factor:
fac_ice = melt_ice / pdd_ice
fac_ice

**Q: compare the two factors. Discuss their relative value in light of the physical properties of snow and ice. Does it make sense for you?**

In [ ]:
# so what do we see?
# for each PDD the lowering of the snow surface is 3 times as much as the ice surface
# would you expect that?
# remember that we are dealing with the rate of surface lowering of 2 different things: snow and ice
# the density of ice is ~900kg/m3, but snow is less dense (even old snow)
# so we might expect the lowering rate to be more per unit of our temperature index

# but what if our calculations were based on ablation in terms of water equivalents or mass instead of surface lowering?
# then the higher reflectance of snow might mean the melt factor for snow is smaller than for ice

**Q: Define a new variable (MIXED_FAC) in the dataframe, which is equal to fac_snow during the snow period and to fac_ice otherwise. Using the same approach as before, compute a new variable MELT2 wich is the cumulative melt during that period. Plot it together with the SR50 observations.**

In [ ]:
# your answer here

df['MIXED_FAC'] = np.where(df['IS_SNOW'], fac_snow, fac_ice)
df['MELT2'] = (df['PDD'] * df['MIXED_FAC']).cumsum() + df.SR50.iloc[0]
df[['MELT1', 'MELT2', 'SR50']].plot(figsize=(10,5));

**Q: Discuss the performance of our new model. Is it performing better than before? Can you tell why?**

In [ ]:
# ok so we have improved it, right?
# but we have not corrected for the snow at the end of our 'ablation' season
# remember these positive changes could be associated with tilting, but if not ...
# then it actually means the ablation season finished earlier than we thought

**Q: The lowest surface height is the 16th August instead of 1st September, how does redefining the ablation season affect our model performance?**

In [ ]:
# reduce dataframe to the true ablation season
df = df.loc['2012-05-15' :'2012-08-16']
#  .... and repeat the whole exercise:

# first, compute the PDD sum during the snowmelt period:
pdd_snow = df.PDD.loc[df['IS_SNOW']].sum()
# then, compute the observed melt during this period:
melt_snow = - df.SR50.iloc[0]
# finally, compute the factor:
fac_snow = melt_snow / pdd_snow
print(fac_snow)
# first, compute the PDD sum during the ice melt period:
pdd_ice = df.PDD.loc[~ df['IS_SNOW']].sum()  # note the ~
# then, compute the observed melt during this period. 
melt_ice = df.SR50.iloc[-1]
# Finally, compute the factor:
fac_ice = melt_ice / pdd_ice
print(fac_ice)
# note the change in the fac_ice value

**Q: Now calculate a new melt series (MELT3) and plot it alongside MELT 2 and the sonic ranger data**

In [ ]:
# make a second version of MIXED_FAC for this new timeframe
# make a cumulative timeseries of modelled melt
# plot it alongside MELT2 and SR50

df['MIXED_FAC2'] = np.where(df['IS_SNOW'], fac_snow, fac_ice)
df['MELT3'] = (df['PDD'] * df['MIXED_FAC2']).cumsum() + df.SR50.iloc[0]
df[['MELT1', 'MELT2', 'MELT3', 'SR50']].plot(figsize=(10,5));

This is a good example of how more perfect information can result in a better performance for this type of empirically fitted model.

**Q: Discuss the following issues in the light of what you did in this exercise:**

- Do we know if these factors are 'right'? 
- Do you expect them to work perfectly in 2013, or any other year, or on another part of the glacier?
- Is it reasonable to expect a melt factor to stay the same over the melt season?
- What else have we not accounted for?


In [ ]:
# your answer here

## 3. An even more reasonable DDM

In our previous models, we compeletely neglected snowfall, which of course is bad. If you are ambitious, you can try to propose solutions to this problem.